### Get CARMA RMS Amp from parameters

<br>**Author(s):** Weixiang Yu
<br>**Last run:** 06-30-2020
<br>**Short description:** This notebook will try to calculate the asymptotic amplitude (or RMS deviation) of the underlying CARMA process from its parameters. 

## 0. Setup

In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# see if local stores mpl style, else use from src
try:
    plt.style.use('yu_basic')
except:
    mpl.rc_file('https://raw.githubusercontent.com/ywx649999311/project_template'
                '/master/%7B%7Bcookiecutter.project_name%7D%7D/src/vis/mpl/yu_basic.rc')

pd.set_option('display.max_columns', 999)
%matplotlib inline

In [2]:
# import kali & Celerite
import kali.carma

import celerite
from celerite import GP, terms
from celerite.solver import get_kernel_value, CARMASolver

In [3]:
def acf(arparam, maparam):
    p = len(arparam)
    q = len(maparam)-1    
    sigma = maparam[0]
    
    # MA param into Kell's notation
    arparam = np.array(arparam)
    maparam = np.array([x/sigma for x in maparam])
    
    # get roots
    arroots = np.roots(np.append([1], arparam))
#     maroots = np.roots(n_maparam[::-1])
    
    # init acf product terms
    num_left = 0
    num_right = 0
    denom = -2*arroots.real + np.zeros_like(arroots)*1j 
    
    for k in range(q+1):
        num_left += maparam[k]*np.power(arroots, k)
        num_right += maparam[k]*np.power(-arroots, k)

    for j in range(1, p):
        root_idx = np.arange(p)
        root_k = arroots[np.roll(root_idx, j)]
        denom *= (root_k - arroots)*(np.conj(root_k) + arroots)

    return sigma**2*num_left*num_right/denom

## 1. Amp from Params

In [6]:
# define function to get amp^2
def get_variance(arparam, maparam):
    acf_ls = acf(arparam, maparam)
    return np.sum(acf_ls)

### 1.1 CARMA(1,0)
Test with:
- $\alpha$ = 1/50, $\sigma$ = 0.1
- $\alpha$ = 1/100, $\sigma$ = 0.01

In [17]:
# get variance from acf
ar_drw_1 = 1/50.0
ma_drw_1 = 0.1
var_from_acf = get_variance([ar_drw_1], [ma_drw_1])

# get variance from kali
task = kali.carma.CARMATask(1,0)
task.set(0.1, np.array([ar_drw_1, ma_drw_1]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

In [18]:
# get variance from acf
ar_drw_1 = 1/100.0
ma_drw_1 = 0.01
var_from_acf = get_variance([ar_drw_1], [ma_drw_1])

# get variance from kali
task = kali.carma.CARMATask(1,0)
task.set(0.1, np.array([ar_drw_1, ma_drw_1]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

### 1.2 CARMA(2,0)
Test with:
- arparam=[2, 1.1], maparam=[0.5]
- arparam=[2, 0.8], maparam=[2] 

In [57]:
# CARMA (2,0)
ar1 = np.array([2, 1.1])
ma1 = np.array([0.5])

# get variance from acf
var_from_acf = get_variance(ar1, ma1)

# get variance from kali
task = kali.carma.CARMATask(2,0)
task.set(0.1, np.concatenate([ar1, ma1]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

In [58]:
ar2 = np.array([2, 0.8])
ma2 = np.array([2])

# get variance from acf
var_from_acf = get_variance(ar2, ma2)

# get variance from kali
task = kali.carma.CARMATask(2,0)
task.set(0.1, np.concatenate([ar2, ma2]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

### 1.3 CARMA(2,1)
Test with:
- arparam=[2, 0.8], maparam=[1, 0.5] 
- arparam=[2, 1.2], maparam=[1, 2]

In [59]:
ar3 = np.array([2, 0.8])
ma3 = np.array([1, 0.5])

# get variance from acf
var_from_acf = get_variance(ar3, ma3)

# get variance from kali
task = kali.carma.CARMATask(2,1)
task.set(0.1, np.concatenate([ar3, ma3]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

In [60]:
ar4 = np.array([2, 1.2])
ma4 = np.array([1, 2])

# get variance from acf
var_from_acf = get_variance(ar4, ma4)

# get variance from kali
task = kali.carma.CARMATask(2,1)
task.set(0.1, np.concatenate([ar4, ma4]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

### 1.4 CARMA(3, 0)
Test with:
- arparam=[3, 2.8, 0.8], maparam=[1] 
- arparam=[3, 3.2, 1.2], maparam=[1]

In [61]:
ar5 = np.array([3, 2.8, 0.8])
ma5 = np.array([1])

# get variance from acf
var_from_acf = get_variance(ar5, ma5)

# get variance from kali
task = kali.carma.CARMATask(3,0)
task.set(0.1, np.concatenate([ar5, ma5]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True

In [62]:
ar6 = np.array([3, 3.2, 1.2])
ma6 = np.array([1])

# get variance from acf
var_from_acf = get_variance(ar6, ma6)

# get variance from kali
task = kali.carma.CARMATask(3,0)
task.set(0.1, np.concatenate([ar6, ma6]))
var_from_kali = task.Sigma()[0][0]

# compare if close
np.isclose(var_from_acf, var_from_kali)

True